In [2]:
# 출처
# https://dooyeoung.tistory.com/36

# 관련문서
# https://developers.google.com/analytics/devguides/reporting/core/v3

# 주요결과 sample 
# https://developers.google.com/analytics/devguides/reporting/core/v3/common-queries

+ **월별 지표**
  + [x] 실적 및 목표(+GA)
  + [x] 고객분석
  + [ ] 광고효율(+GA)
  + [x] 품목별 매출


+ **주별 지표**
  + [ ] 실적 및 목표(+GA)
  + [ ] 고객분석
  + [ ] 광고효율(+GA)
  + [ ] 품목별 매출

In [1]:
# !pip install --upgrade google-api-python-client

In [1]:
import cx_Oracle as oci

from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

import numpy as np
import pandas as pd

from datetime import datetime


In [2]:
def get_service(api_name, api_version, scopes, key_file_location):
    credentials = ServiceAccountCredentials.from_json_keyfile_name(key_file_location, scopes=scopes)
    service = build(api_name, api_version, credentials=credentials)
    return service
    
# Define the auth scopes to request.
scope = 'https://www.googleapis.com/auth/analytics.readonly'
key_file_location = './vision API-06a448b64428.json'

# Authenticate and construct service.
service = get_service(api_name='analytics', api_version='v3', scopes=[scope], key_file_location=key_file_location)

# Get a list of all Google Analytics accounts for this user
accounts = service.management().accounts().list().execute()
if accounts.get('items'):
    # Get the first Google Analytics account.
    account = accounts.get('items')[0].get('id')
    # Get a list of all the properties for the first account.
    properties = service.management().webproperties().list(accountId=account).execute()

In [3]:
# 조건1 : 일자
start = "2021-01-01"
end = datetime.today().strftime('%Y-%m-%d')
start_date_info = [d.strftime('%Y-%m-%d') for d in pd.date_range(start, end, freq='MS')]
end_date_info = [d.strftime('%Y-%m-%d') for d in pd.date_range(start, end, freq='M')]
if len(start_date_info) != len(end_date_info):
    end_date_info.append(end)

## 월별지표

### 1.실적 및 목표(GA)

+ [x] 방문자수 : users
+ [x] 세션수 : sessions
+ [x] 방문당 세션수 : sessionsPerUser
+ [x] 신규방문자수 : newUsers



In [4]:
##########
# 자료확보 : GA
####

# 조건2 : 필터링 항목(dimensions)

# 조건3 : 측정 단위(metrics)
metrics = [ 'ga:users', 'ga:sessions', 'ga:sessionsPerUser', 'ga:newUsers']
df = pd.DataFrame()

for metrics_value in metrics:
    # 수집내용 정의
    column_name = (metrics_value).replace('ga:', '').split(",")
    df2_ga = pd.DataFrame( columns =['CNT'])
    df2_ga['date'] = ''

    # 결과 수집+정리
    for i in range(0, len(start_date_info)):
        try:
            result = service.data().ga().get(ids='ga:236972743',
                                             start_date=start_date_info[i], end_date=end_date_info[i], metrics=metrics_value).execute()
            df_temp = pd.DataFrame(result['rows'], columns =['CNT'])
            df_temp['date'] = start_date_info[i]
            df2_ga = pd.concat([df2_ga, df_temp])
        except:
            pass

    # 자료의 상위속성 추가
    df2_ga['gubun'] = column_name[0]
    df = pd.concat([df2_ga, df], ignore_index=True)

In [5]:
# df

In [6]:
# 임시자료 추가(2021년 1월 미수집에 따른)
df_temp = pd.DataFrame({"CNT":[0], "date":['2021-01'], "gubun":['users']})
df = df.append(df_temp, ignore_index = True)

In [7]:
# 월 정렬
df['date'] = df['date'].str[:7]
df_MTLY = pd.pivot_table(df, index = ['gubun'], values = 'CNT', columns = 'date', aggfunc = np.sum).fillna(0)

# 기준 테이블 작성/order 수정
df_order = pd.DataFrame()
df_order['gubun'] = ['users', 'sessions', 'sessionsPerUser', 'newUsers']
df_MTLY= pd.merge(df_order, df_MTLY, on ='gubun', how ='left')
df_MTLY.set_index(df_MTLY.gubun,inplace = True)
del df_MTLY['gubun']
df_MTLY = df_MTLY.T
df_MTLY.rename(columns = {'users' : '방문자수', 'sessions' : '세션수',
                          'sessionsPerUser' : '방문당 세션수','newUsers' : '신규방문자수'}, inplace = True)

df_MTLY['신규 방문 비중(%)'] = df_MTLY['신규방문자수'].astype(int)/df_MTLY['방문자수'].astype(int)
# df_MTLY = df_MTLY.T.fillna(0)

In [8]:
# df_MTLY.index

In [9]:
df_MTLY

gubun,방문자수,세션수,방문당 세션수,신규방문자수,신규 방문 비중(%)
2021-01,0,0,0,0,NaN
2021-02,19729,52242,2.64798013077196,19874,1.007350
2021-03,24133,45195,1.8727468611444908,20231,0.838313
2021-04,121000,171801,1.4198429752066115,116726,0.964678
2021-05,170289,260373,1.5290065711819318,153716,0.902677
2021-06,174456,414592,2.3764846150318704,151574,0.868838
2021-07,14731,24556,1.6669608309008215,9543,0.647818


In [10]:
##########
# 자료확보 : Oracle
####

# query 열기
query = "C:/Users/MAEIL/Desktop/셀렉스몰/18. query(AARRR)/AARRR_query_v3/1.report.txt"

# DB접속 : 접속정보 읽기
con_text = open("C:/Users/MAEIL/conn_SELEX.txt", 'r', encoding='utf8')
con_text = con_text.read()

# DB 접속 : 실제 접속
conn = oci.connect(con_text[1:], encoding='UTF-8', nencoding='UTF-8')

f = open(query, 'r')
line = f.read()
f.close()

## query 실행( SQL문 실행 메모리 영역에서 진행)
cursor = conn.cursor()
cursor.execute(line)
df1 = cursor.fetchall()[1:]

df1 = pd.DataFrame(df1, columns = ['MONTH', '신규회원수', '회원가입전환율(%)',
                                   '누적회원수', '주문건수', '주문전환율(%)',
                                   '주문금액', '사용한적립금', '사용한적립금(%)', 
                                   '주문단가', '결제금액', 
                                   '유선주문건','유선주문금액', '유선주문금액(%)',
                                   '첫구매인원'])

In [11]:
df1.set_index(df1.MONTH,inplace = True)
del df1['MONTH']

In [12]:
# df1

In [13]:
df1['회원가입전환율(%)'] = df1['신규회원수'] / df_MTLY.신규방문자수.astype(int)
df1['주문전환율(%)'] = df1['주문건수'] / df_MTLY.신규방문자수.astype(int)
# df1['결제금액'] = df1['주문건수'] / df_MTLY.신규방문자수.astype(int)

In [14]:
df1_all = pd.concat([df_MTLY.T, df1.T]).fillna(0)

In [15]:
# df1_all.fillna(0)

In [16]:
df1_all = df1_all.astype(float)


In [17]:
df1_all

,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07
방문자수,0.000000e+00,1.972900e+04,2.413300e+04,1.210000e+05,1.702890e+05,1.744560e+05,1.473100e+04
세션수,0.000000e+00,5.224200e+04,4.519500e+04,1.718010e+05,2.603730e+05,4.145920e+05,2.455600e+04
방문당 세션수,0.000000e+00,2.647980e+00,1.872747e+00,1.419843e+00,1.529007e+00,2.376485e+00,1.666961e+00
신규방문자수,0.000000e+00,1.987400e+04,2.023100e+04,1.167260e+05,1.537160e+05,1.515740e+05,9.543000e+03
신규 방문 비중(%),0.000000e+00,1.007350e+00,8.383127e-01,9.646777e-01,9.026772e-01,8.688380e-01,6.478175e-01
신규회원수,8.880000e+02,2.033500e+04,1.788000e+03,3.634000e+03,6.492000e+03,5.871000e+03,9.100000e+02
회원가입전환율(%),inf,1.023196e+00,8.837922e-02,3.113274e-02,4.223373e-02,3.873356e-02,9.535785e-02
누적회원수,8.940000e+02,2.122900e+04,2.301700e+04,2.665100e+04,3.314300e+04,3.901400e+04,3.992400e+04
주문건수,5.380000e+02,1.568400e+04,2.963000e+03,4.097000e+03,6.084000e+03,5.915000e+03,7.410000e+02
주문전환율(%),inf,7.891718e-01,1.464584e-01,3.509929e-02,3.957948e-02,3.902384e-02,7.764854e-02


### 2.고객분석

In [18]:
##########
# 자료확보 : Oracle
####

# query 열기
query = "C:/Users/MAEIL/Desktop/셀렉스몰/18. query(AARRR)/AARRR_query_v3/2.report_by_user.txt"

# DB접속 : 접속정보 읽기
con_text = open("C:/Users/MAEIL/conn_SELEX.txt", 'r', encoding='utf8')
con_text = con_text.read()

# DB 접속 : 실제 접속
conn = oci.connect(con_text[1:], encoding='UTF-8', nencoding='UTF-8')

f = open(query, 'r')
line = f.read()
f.close()

## query 실행( SQL문 실행 메모리 영역에서 진행)
cursor = conn.cursor()
cursor.execute(line)
df2 = cursor.fetchall()[1:]

df2 = pd.DataFrame(df2, columns = ['MONTH', 'GUBUN', '주문고객수', '주문건수', '주문금액', '적립금사용', '매출액'])
# df2 = pd.DataFrame(df2, columns = ['ID', 'MONTH', 'cum_count', '주문건수', '매출액', '적립금사용', '주문금액'])

In [19]:
df2.head()

,MONTH,GUBUN,주문고객수,주문건수,주문금액,적립금사용,매출액
0,2021-01,1,389,389,15115910,3952800,11163110
1,2021-02,1,14344,14344,355777150,168794540,186982610
2,2021-03,1,1639,1639,72413772,7519830,64893942
3,2021-04,1,2499,2499,100694030,10943000,89751030
4,2021-05,1,3861,3861,174700970,17560800,157140170


In [20]:
# 기준 테이블 작성
df2_order = pd.DataFrame()
df2_order['GUBUN'] = ['주문고객수', '주문건수', '주문금액', '적립금사용', '매출액']
df2_order = df2_order.set_index('GUBUN')

In [21]:
# 유형별 자료 정리
df2_gubun1 = pd.pivot_table(df2[df2['GUBUN']== 1] ,
                            values = ['주문고객수','주문건수','주문금액','적립금사용','매출액'], 
                            index='MONTH', aggfunc='sum').T
df2_gubun1 = pd.merge(df2_order, df2_gubun1, left_index= True,right_index=True, how='left' )
df2_gubun1['GUBUN2'] = 1

df2_gubun2 = pd.pivot_table(df2[df2['GUBUN']== 2] ,
                            values = ['주문고객수','주문건수','주문금액','적립금사용','매출액'], 
                            index='MONTH', aggfunc='sum').T
df2_gubun2 = pd.merge(df2_order, df2_gubun2, left_index= True,right_index=True, how='left' )
df2_gubun2['GUBUN2'] = 2

df2_gubun3 = pd.pivot_table(df2[df2['GUBUN']== 3],
                            values = ['주문고객수','주문건수','주문금액','적립금사용','매출액'], 
                            index='MONTH', aggfunc='sum').T
df2_gubun3 = pd.merge(df2_order, df2_gubun3, left_index= True,right_index=True, how='left' )
df2_gubun3['GUBUN2'] = 3

df2_total = df2_gubun1 + df2_gubun2 + df2_gubun3

In [22]:
# 자료 합치기
df2_all = pd.concat([df2_total, df2_gubun1,df2_gubun2, df2_gubun3 ])

In [23]:
df2_all

,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,GUBUN2
GUBUN,,,,,,,,
주문고객수,540,17076,3014,4146,5790,5360,1224,6
주문건수,603,17554,3239,4523,6676,6547,1372,6
주문금액,24615370,443965630,139154252,186464330,326212740,364789450,75428300,6
적립금사용,4825390,200590830,13528240,18649800,24939640,11576950,2657870,6
매출액,19789980,243374800,125626012,167814530,301273100,353212500,72770430,6
주문고객수,389,14344,1639,2499,3861,3060,666,1
주문건수,389,14344,1639,2499,3861,3060,666,1
주문금액,15115910,355777150,72413772,100694030,174700970,161041260,34611360,1
적립금사용,3952800,168794540,7519830,10943000,17560800,7733100,1828990,1


### 3.품목별 매출

In [24]:
##########
# 자료확보 : Oracle
####

# query 열기
query = "C:/Users/MAEIL/Desktop/셀렉스몰/18. query(AARRR)/AARRR_query_v3/3.report_by_product.txt"

# DB접속 : 접속정보 읽기
con_text = open("C:/Users/MAEIL/conn_SELEX.txt", 'r', encoding='utf8')
con_text = con_text.read()

# DB 접속 : 실제 접속
conn = oci.connect(con_text[1:], encoding='UTF-8', nencoding='UTF-8')

f = open(query, 'r')
line = f.read()
f.close()

## query 실행( SQL문 실행 메모리 영역에서 진행)
cursor = conn.cursor()
cursor.execute(line)
df3 = cursor.fetchall()[1:]

df3 = pd.DataFrame(df3, columns = ['MONTH', 'GUBUN', '주문수량', '결재금액'])
# df2 = pd.DataFrame(df2, columns = ['ID', 'MONTH', 'cum_count', '주문건수', '매출액', '적립금사용', '주문금액'])

In [25]:
df3

,MONTH,GUBUN,주문수량,결재금액
0,2021-01,상품 전체보기 < 밀크세라마이드 콜라겐,176,4661589
1,2021-01,상품 전체보기 < 슬림25 다이어트 쉐이크,30,785340
2,2021-01,상품 전체보기 < 웨이프로틴 드링크,13,504230
3,2021-01,상품 전체보기 < 웨이프로틴 파우더,41,1076577
4,2021-01,상품 전체보기 < 정기배송,2,47880
...,...,...,...,...
90,2021-07,상품 전체보기 < 코어프로틴,832,29292957
91,2021-07,상품 전체보기 < 프로틴 바,65,1583816
92,2021-07,시크릿 정기배송 특가,1,40320
93,2021-07,임직원관,2,142400


In [26]:
df3_count  = pd.pivot_table(df3, values = '주문수량', index='GUBUN', columns='MONTH',aggfunc='sum')
df3_amount = pd.pivot_table(df3, values = '결재금액', index='GUBUN', columns='MONTH',aggfunc='sum')

In [27]:
df3_count.fillna(0)

MONTH,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07
GUBUN,,,,,,,
상담사관,0.0,0.0,0.0,25.0,506.0,787.0,112.0
상품 전체보기 < 마시는 프로틴,0.0,5909.0,1469.0,872.0,899.0,947.0,331.0
상품 전체보기 < 밀크세라마이드 콜라겐,176.0,15145.0,268.0,1425.0,1347.0,1214.0,78.0
상품 전체보기 < 선물세트,0.0,0.0,0.0,96.0,306.0,89.0,22.0
상품 전체보기 < 슬림25 다이어트 쉐이크,30.0,1260.0,856.0,937.0,812.0,401.0,90.0
상품 전체보기 < 웨이프로틴 드링크,13.0,282.0,326.0,226.0,422.0,475.0,63.0
상품 전체보기 < 웨이프로틴 파우더,41.0,2244.0,866.0,1622.0,1869.0,1327.0,205.0
상품 전체보기 < 정기배송,2.0,15.0,45.0,121.0,357.0,500.0,89.0
상품 전체보기 < 체험키트,9.0,260.0,417.0,1643.0,843.0,236.0,38.0


In [28]:
df3_amount.fillna(0)

MONTH,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07
GUBUN,,,,,,,
상담사관,0.0,0.0,0.0,766000.0,41366100.0,58474500.0,7043900.0
상품 전체보기 < 마시는 프로틴,0.0,127498253.0,39249965.0,36531527.0,38770510.0,38294500.0,12960817.0
상품 전체보기 < 밀크세라마이드 콜라겐,4661589.0,153257289.0,7126327.0,32276077.0,32139748.0,32797834.0,2374316.0
상품 전체보기 < 선물세트,0.0,0.0,0.0,2573950.0,8680406.0,2443628.0,611583.0
상품 전체보기 < 슬림25 다이어트 쉐이크,785340.0,15120100.0,8470510.0,9662178.0,10949500.0,6307851.0,1343872.0
상품 전체보기 < 웨이프로틴 드링크,504230.0,8723499.0,9433381.0,9425324.0,17013200.0,18351889.0,2442428.0
상품 전체보기 < 웨이프로틴 파우더,1076577.0,28230962.0,14469481.0,22630517.0,41361768.0,36241704.0,5908704.0
상품 전체보기 < 정기배송,47880.0,661600.0,2109742.0,5509640.0,15861220.0,23454930.0,4070560.0
상품 전체보기 < 체험키트,75600.0,1747600.0,2384636.0,5059467.0,6898100.0,1893284.0,257843.0


In [30]:
##########
# Excel 파일 생성(using XlsxWriter)
###

date_today = datetime.today().strftime('%Y%m%d')
date_today = date_today[2:]
excel_name = date_today +'_사업부_운영현황_Monthly.xlsx'
writer = pd.ExcelWriter(excel_name, engine='xlsxwriter')

# 1.실적
df1_all.to_excel(writer, sheet_name='1.실적')

# 2.버즈량 저장
df2_all.to_excel(writer, sheet_name='2.고객분석')

# 3.품목별 매출
df3_count.to_excel(writer, sheet_name='3.품목(수량)')
df3_amount.to_excel(writer, sheet_name='3.품목(매출)')

# Close the Pandas Excel writer and output the Excel file.
writer.save()